# Notebook for visualizations

In [73]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import row, layout
from bokeh.models import ColumnDataSource
from bokeh.transform import factor_cmap, factor_mark
from IPython.display import display, Math
from scipy import optimize
from math import exp, cos, sqrt, pi, sin, sqrt
from typing import List

from src.OspitalettoDataset import OspitalettoDataset
from src.NOAA2010Dataset import NOAA2010Dataset
from src.InsPireDataset import InsPireDataset

mypalette = sns.color_palette(['#51ffda']) # https://www.hsluv.org/
sns.set_palette(mypalette)

output_notebook()

Loading BokehJS ...

In [2]:
ospitaletto2019 = OspitalettoDataset()
noaa2010Dataset = NOAA2010Dataset()
insPireDataset = InsPireDataset()

AVAILABLE_DATASETS = dict()
AVAILABLE_DATASETS.update(ospitaletto2019.load_processed_data())
AVAILABLE_DATASETS.update(noaa2010Dataset.load_processed_data())
AVAILABLE_DATASETS.update(insPireDataset.load_processed_data())
AVAILABLE_DATASETS.keys()

dict_keys(['ospitaletto', 'miami_florida', 'fresno_california', 'olympia_washington', 'rochester_newyork', 'london_uk', 'madrid_spain', 'rome_italy', 'stuttgart_germany'])

## Important visualization functions

In [3]:
def visualize_on_same_row(data_list: List, ylabel: str, xlabel: str, suptitle: str):
    fig, axs = plt.subplots(nrows=1, ncols=5, sharey=True, figsize=(40,8), subplot_kw={"ylabel": ylabel, "xlabel": xlabel})
    
    axs[0].set_title('Miami, FL, USA')
    sns.lineplot(data=data_list[0], ax=axs[0])

    axs[1].set_title('Fresno, CA, USA')
    sns.lineplot(data=data_list[1], ax=axs[1])

    axs[2].set_title('Olympia, WA, USA')
    sns.lineplot(data=data_list[2], ax=axs[2])

    axs[3].set_title('Rochester, NY, USA')
    sns.lineplot(data=data_list[3], ax=axs[3])

    axs[4].set_title('Ospitaletto, Italy')
    sns.lineplot(data=data_list[4], ax=axs[4])

    fig.suptitle(suptitle)
    
    return fig, axs
    

## Visualization of curve fitting

In [4]:
ospitaletto = AVAILABLE_DATASETS[OspitalettoDataset.OSPITALETTO]

miami = AVAILABLE_DATASETS[NOAA2010Dataset.MIAMI_FL]
fresno = AVAILABLE_DATASETS[NOAA2010Dataset.FRESNO_CA]
olympia = AVAILABLE_DATASETS[NOAA2010Dataset.OLYMPIA_WA]
rochester = AVAILABLE_DATASETS[NOAA2010Dataset.ROCHESTER_NY]

london = AVAILABLE_DATASETS[InsPireDataset.LONDON_UK]
madrid = AVAILABLE_DATASETS[InsPireDataset.MADRID_SPA]
rome = AVAILABLE_DATASETS[InsPireDataset.ROME_IT]
stuttgart = AVAILABLE_DATASETS[InsPireDataset.STUTTGART_GER]

In [91]:
ospitaletto["city"] = "ospitaletto"

miami["city"] = "miami"
fresno["city"] = "fresno"
olympia["city"] = "olympia"
rochester["city"] = "rochester"
us_data = miami.append(other=fresno).append(other=olympia).append(other=rochester)

london["city"] = "london"
madrid["city"] = "madrid"
rome["city"] = "rome"
stuttgart["city"] = "stuttgart"
europe_data = london.append(other=madrid).append(other=rome).append(other=stuttgart)

In [94]:
source_usa = ColumnDataSource({"timestamp": miami.index,
                                 
                                 "miami_air_temp": miami.air_temp,
                                 "miami_air_temp_fit": miami.air_temp_fit,
                                 
                                 "fresno_air_temp": fresno.air_temp,
                                 "fresno_air_temp_fit": fresno.air_temp_fit,
                                 
                                 "olympia_air_temp": olympia.air_temp,
                                 "olympia_air_temp_fit": olympia.air_temp_fit,
                                 
                                 "rochester_air_temp": rochester.air_temp,
                                 "rochester_air_temp_fit": rochester.air_temp_fit,
                                })

max_temp = int(us_data.air_temp.max() + 5)
min_temp = int(us_data.air_temp.min() - 5)

miami_air_temp_and_fit = figure(plot_width=600, 
                                plot_height=300, 
                                x_axis_type="datetime",
                                title="Miami, Florida, USA",
                                y_range=(min_temp, max_temp),)
miami_air_temp_and_fit.line(x="timestamp", y="miami_air_temp", legend_label="Air Temp", color='navy', alpha=0.2, source=source_usa)
miami_air_temp_and_fit.circle(x="timestamp", y="miami_air_temp_fit", legend_label="Fit Air Temp", color='navy', alpha=0.1, source=source_usa, radius=1)

fresno_air_temp_and_fit = figure(plot_width=600,  
                                 plot_height=300, 
                                 x_axis_type="datetime",
                                title="Fresno, California, USA",
                                y_range=miami_air_temp_and_fit.y_range,
                                x_range=miami_air_temp_and_fit.x_range)
fresno_air_temp_and_fit.line(x="timestamp", y="fresno_air_temp", legend_label="Air Temp", color='red', alpha=0.2, source=source_usa)
fresno_air_temp_and_fit.circle(x="timestamp", y="fresno_air_temp_fit", legend_label="Fit Air Temp", color='red', alpha=0.1, source=source_usa, radius=1)

olympia_air_temp_and_fit = figure(plot_width=600, 
                                  plot_height=300, 
                                  x_axis_type="datetime",
                                title="Olympia, Washington, USA",
                                y_range=miami_air_temp_and_fit.y_range,
                                x_range=miami_air_temp_and_fit.x_range)
olympia_air_temp_and_fit.line(x="timestamp", y="olympia_air_temp", legend_label="Air Temp", color='green', alpha=0.2, source=source_usa)
olympia_air_temp_and_fit.circle(x="timestamp", y="olympia_air_temp_fit", legend_label="Fit Air Temp", color='green', alpha=0.1, source=source_usa, radius=1)

rochester_air_temp_and_fit = figure(plot_width=600, 
                                    plot_height=300, 
                                    x_axis_type="datetime",
                                title="Rochester, New York, USA",
                                y_range=miami_air_temp_and_fit.y_range,
                                x_range=miami_air_temp_and_fit.x_range)
rochester_air_temp_and_fit.line(x="timestamp", y="rochester_air_temp", legend_label="Air Temp", color='gray', alpha=0.2, source=source_usa)
rochester_air_temp_and_fit.circle(x="timestamp", y="rochester_air_temp_fit", legend_label="Fit Air Temp", color='gray', alpha=0.1, source=source_usa, radius=1)


only_air_temp_fit = figure(plot_width=1200, 
                           plot_height=300, 
                           x_axis_type="datetime",
                                title="Adjusted air temperature",
                          y_range=miami_air_temp_and_fit.y_range,
                                x_range=miami_air_temp_and_fit.x_range)
only_air_temp_fit.line(x="timestamp", y="miami_air_temp_fit", legend_label="Miami", color='navy', alpha=0.7, source=source_usa)
only_air_temp_fit.line(x="timestamp", y="fresno_air_temp_fit", legend_label="Fresno", color='red', alpha=0.7, source=source_usa)
only_air_temp_fit.line(x="timestamp", y="olympia_air_temp_fit", legend_label="Olympia", color='green', alpha=0.7, source=source_usa)
only_air_temp_fit.line(x="timestamp", y="rochester_air_temp_fit", legend_label="Rochester", color='gray', alpha=0.7, source=source_usa)

show(layout([
    [miami_air_temp_and_fit, fresno_air_temp_and_fit,],
    [olympia_air_temp_and_fit, rochester_air_temp_and_fit],
    [only_air_temp_fit],
]))

In [95]:
source_europe = ColumnDataSource({"timestamp": london.index,
                                 
                                 "london_air_temp": london.air_temp,
                                 "london_air_temp_fit": london.air_temp_fit,
                                 
                                 "madrid_air_temp": madrid.air_temp,
                                 "madrid_air_temp_fit": madrid.air_temp_fit,
                                 
                                 "rome_air_temp": rome.air_temp,
                                 "rome_air_temp_fit": rome.air_temp_fit,
                                 
                                 "stuttgart_air_temp": stuttgart.air_temp,
                                 "stuttgart_air_temp_fit": stuttgart.air_temp_fit,
                                })

max_temp = int(europe_data.air_temp.max() + 5)
min_temp = int(europe_data.air_temp.min() - 5)

london_air_temp_and_fit = figure(plot_width=600, 
                                plot_height=300, 
                                x_axis_type="datetime",
                                title="London, UK",
                                y_range=(min_temp, max_temp),)
london_air_temp_and_fit.line(x="timestamp", y="london_air_temp", legend_label="Air Temp", color='navy', alpha=0.2, source=source_europe)
london_air_temp_and_fit.circle(x="timestamp", y="london_air_temp_fit", legend_label="Fit Air Temp", color='navy', alpha=0.1, source=source_europe, radius=1)

madrid_air_temp_and_fit = figure(plot_width=600,  
                                 plot_height=300, 
                                 x_axis_type="datetime",
                                title="Madrid, Spain",
                                y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
madrid_air_temp_and_fit.line(x="timestamp", y="madrid_air_temp", legend_label="Air Temp", color='red', alpha=0.2, source=source_europe)
madrid_air_temp_and_fit.circle(x="timestamp", y="madrid_air_temp_fit", legend_label="Fit Air Temp", color='red', alpha=0.1, source=source_europe, radius=1)

rome_air_temp_and_fit = figure(plot_width=600, 
                                  plot_height=300, 
                                  x_axis_type="datetime",
                                title="Rome, Italy",
                                y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
rome_air_temp_and_fit.line(x="timestamp", y="rome_air_temp", legend_label="Air Temp", color='green', alpha=0.2, source=source_europe)
rome_air_temp_and_fit.circle(x="timestamp", y="rome_air_temp_fit", legend_label="Fit Air Temp", color='green', alpha=0.1, source=source_europe, radius=1)

stuttgart_air_temp_and_fit = figure(plot_width=600, 
                                    plot_height=300, 
                                    x_axis_type="datetime",
                                title="Stuttgart, Germany",
                                y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
stuttgart_air_temp_and_fit.line(x="timestamp", y="stuttgart_air_temp", legend_label="Air Temp", color='gray', alpha=0.2, source=source_europe)
stuttgart_air_temp_and_fit.circle(x="timestamp", y="stuttgart_air_temp_fit", legend_label="Fit Air Temp", color='gray', alpha=0.1, source=source_europe, radius=1)


only_air_temp_fit = figure(plot_width=1200, 
                           plot_height=300, 
                           x_axis_type="datetime",
                                title="Adjusted air temperature",
                          y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
only_air_temp_fit.line(x="timestamp", y="london_air_temp_fit", legend_label="London", color='navy', alpha=0.7, source=source_europe)
only_air_temp_fit.line(x="timestamp", y="madrid_air_temp_fit", legend_label="madrid", color='red', alpha=0.7, source=source_europe)
only_air_temp_fit.line(x="timestamp", y="rome_air_temp_fit", legend_label="rome", color='green', alpha=0.7, source=source_europe)
only_air_temp_fit.line(x="timestamp", y="stuttgart_air_temp_fit", legend_label="stuttgart", color='gray', alpha=0.7, source=source_europe)

show(layout([
    [london_air_temp_and_fit, stuttgart_air_temp_and_fit ,],
    [rome_air_temp_and_fit, madrid_air_temp_and_fit],
    [only_air_temp_fit],
]))

## Visualization of Air Temp, Ground Temp and Aquifer Temp

In [ ]:
visualize_on_same_row(data_list=[miami[["air_temp", "ground_temp", "aquifer_temp"]],
                                 fresno[["air_temp", "ground_temp", "aquifer_temp"]],
                                 olympia[["air_temp", "ground_temp", "aquifer_temp"]],
                                 rochester[["air_temp", "ground_temp", "aquifer_temp"]],
                                 ospitaletto[["air_temp", "ground_temp", "aquifer_temp"]]],
                      ylabel="Temperature in Celsius", 
                      xlabel="Date",
                      suptitle='Air, ground and aquifer temperature in different locations')

## Visualization of Hot Water and Space Heating temperatures

### Whole year

In [ ]:
visualize_on_same_row(data_list=[miami[["space_heating_temp","hot_water_temp"]],
                                 fresno[["space_heating_temp","hot_water_temp"]],
                                 olympia[["space_heating_temp","hot_water_temp"]],
                                 rochester[["space_heating_temp","hot_water_temp"]],
                                 ospitaletto[["space_heating_temp","hot_water_temp"]]],
                      ylabel="Temperature in Celsius", 
                      xlabel="Date",
                      suptitle='Calculated Space Heating and Hot Water temperatures')

In [ ]:
month = int(input("Enter month to analyze (values between 1 to 12): "))

In [ ]:
november_data = Tamb_hourly[Tamb_hourly.month == month ]
november_data[["space_heating_temp","hot_water_temp"]].plot(figsize=(10,6), color=["#4e97a2",'#82272e'])

plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

In [ ]:
Tamb_hourly["user_temp"].plot(figsize=(10,6))

In [ ]:
november_data = Tamb_hourly[Tamb_hourly.index.month == 11 ]
november_data["Tuser"].plot(figsize=(10,6))

## Visualization of Net temperature

In [ ]:
Tamb_hourly[["net_temp"]][Tamb_hourly.index < "2010-01-31"].plot()

## Visualization of COP

In [ ]:
Tamb_hourly[['COP']].plot(figsize=(20,6))

## Visualization of Energy Loss

In [ ]:
Tamb_hourly[['E_loss_tot']].plot(figsize=(20,6))

## Space Heating Distribution Plot

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(10,10))

# Stuttgart

sns.lineplot(data=stuttgart[["space_heating_dist"]], ax=ax1, color='#4d4dff')
ax1.set_title('Stuttgart')
ax1.set_ylabel('Space heating hourly distribution')
ax1.grid()

# London
sns.lineplot(data=london[["space_heating_dist"]], ax=ax2, color='#8686ef')
ax2.set_title('London')
ax2.grid()

# Madrid
sns.lineplot(data=madrid[["space_heating_dist"]], ax=ax3, color='#ef8e49')
ax3.set_title('Madrid')
ax3.set_ylabel('Space heating hourly distribution')
ax3.grid()

# Rome
sns.lineplot(data=rome[["space_heating_dist"]], ax=ax4, color= '#ca0c6e')
ax4.set_title('Rome')
ax4.grid()

fig.tight_layout()
plt.show()

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(10,10))

# Stuttgart

sns.lineplot(data=stuttgart[["space_heating_dist"]], ax=ax1, color='#4d4dff')
ax1.set_title('Stuttgart')
ax1.set_ylabel('Space heating hourly distribution')
ax1.grid()

# London
sns.lineplot(data=london[["space_heating_dist"]], ax=ax2, color='#8686ef')
ax2.set_title('London')
ax2.grid()

# Madrid
sns.lineplot(data=madrid[["space_heating_dist"]], ax=ax3, color='#ef8e49')
ax3.set_title('Madrid')
ax3.set_ylabel('Space heating hourly distribution')
ax3.grid()

# Rome
sns.lineplot(data=rome[["space_heating_dist"]], ax=ax4, color= '#ca0c6e')
ax4.set_title('Rome')
ax4.grid()

fig.tight_layout()
plt.show()